In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
data_file = './kddcup.data_10_percent.gz'
raw_data = sc.textFile(data_file)

In [8]:
# parse data
csv_data = raw_data.map(lambda x: x.split(','))

# separate into different RDDs
normal_csv_data = csv_data.filter(lambda x: x[41] == 'normal.')
attack_csv_data = csv_data.filter(lambda x: x[41] != 'normal.')

In [13]:
normal_duration_data = normal_csv_data.map(lambda x: int(x[0]))
attack_duration_data = attack_csv_data.map(lambda x: int(x[0]))

In [14]:
# calculate total durations
total_normal_duration = normal_duration_data.reduce(lambda x, y: x + y)
total_attack_duration = attack_duration_data.reduce(lambda x, y: x + y)
print("Total duration for normal interactions is {}".format(total_normal_duration))
print("Total duration for attack interactions is {}".format(total_attack_duration))

Total duration for normal interactions is 21075991
Total duration for attack interactions is 2626792


In [19]:
# calculation duration means
normal_count = normal_duration_data.count()
attack_count = attack_duration_data.count()
print("Mean duration for normal interaction is {}".format(round(total_normal_duration / normal_count, 3)))
print("Mean duration for attack interaction is {}".format(round(total_attack_duration / attack_count, 3)))

Mean duration for normal interaction is 216.657
Mean duration for attack interaction is 6.621


In [20]:
normal_duration_data.take(5)

[0, 0, 0, 0, 0]

In [21]:
normal_sum_count = normal_duration_data.aggregate(
    (0, 0),   # the inital value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
)

print("Mean duration for normal interaction is {}".format(round(normal_sum_count[0] / normal_sum_count[1], 3)))

Mean duration for normal interaction is 216.657
